Pra instalar o Jupyter é so seguir a descricao do video (la ele ensina o basicao dessa IDE): https://www.youtube.com/watch?v=_eK0z5QbpKA&t=926s

Before running the code it is important to download the chrome webdriver: https://chromedriver.chromium.org/downloads
Open the location in which Python is installed, for that open Anaconda prompt. In my case "C:\ProgramData\Anaconda3"

In [3]:
#Packages to be installed (run only once)

#This package is used to automate tasks in chrome
!pip install selenium
#This package is used to automated keyboard and mouse functions
!pip install pyautogui
#This package is used to copy and paste special characters
!pip install pyperclip
#This package is used to plot the data
!pip install plotly

The next part will import the valeu of the euro in relation to reais from Google

In [10]:
#Import Selenium package (https://selenium-python.readthedocs.io/)
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#Import pyautogui to control the keyboard and the mouse
import pyautogui

#Import pyperclip to copy and paste special characters
import pyperclip

#Import time to control timing
import time

#Make the code waits x seconds between each comand line
pyautogui.PAUSE = 0.5

#Open a new window in Chrome
new_tab = webdriver.Chrome()
#Open an specific website
new_tab.get("https://www.google.com.br/")

#Access an specific location of the site and type a message
#To find the location of the website you want, right click on it, click on "investigate", type "ctrl + shift + c", click on the part you want (it will be highlighted in blue on the right side), right click on the highlighted blue region -> copy -> copy xpath
new_tab.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('1 Euro in Reais')
#Access the same location and press enter
new_tab.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

#Getting the wanted result
#To find the attribute you want just inspect the page like before but instead of copying the xpath read the highlighted blue region and check the data you want
euro_in_reais_str = new_tab.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
euro_in_reais= float(euro_in_reais_str)

This part of the code import an excel table and clean the data

In [5]:
#Import the math lybrary
import pandas as pd

#Read the data inside the table (press tab after '.' to check all the methods)
new_table = pd.read_excel("Sales.xlsx")

#To see the table content
display(new_table)

#Prints the summarized information of the table
print(new_table.info())

#Deletes an entire column (axis=1) with the title ("Unnamed: 0")
new_table = new_table.drop("Unnamed: 0", axis=1)

#The values in "TotalGasto" were read as string values, 
#to correct that they were changed using ".to_numeric", 
#the parameter "errors="coerce"" forces the numbers that cant be 
#transformed in numbers to get Nan
new_table[r"Total expends (R$)"] = pd.to_numeric(new_table[r"Total expends (R$)"], errors="coerce")

#To automatic delete columns that are empty the command ".dropna" is used
#how is a parameters that chooses to delete columns (axis=1) 
#that are completely empty (how="all")
new_table = new_table.dropna(how="all", axis=1)

#To automatic delete columns that are empty the command ".dropna" is used
#how is a parameters that chooses to delete lines (axis=0) 
#that have at least one parameter which is empty (how="any")
new_table = new_table.dropna(how="any", axis=0)

#The command ".value_counts()" counts each value inside the chosen column ("Churn")
display(new_table["Childs"].value_counts())

#The command ".value_counts()" counts each value inside the chosen column ("Churn")
#"(normalize=True)" print the values in percentage
#".map("{:.1%}".format)" formats the code as percentage with one decimal place
display(new_table["Childs"].value_counts(normalize=True).map("{:.1%}".format))

#Update the values of the total expenses
new_table["Total expends (e)"] = new_table["Total expends (R$)"] / euro_in_reais

#To see the table content after treatment and updated values for the total expenses in euros
display(new_table)

#Save new_table as csv
new_table.to_csv("Updated_Sales.csv")

#Sum the total expenses in euros and reais
total_reais_expenses =  new_table["Total expends (R$)"].sum()
print(f'The total amount expended in reais was {total_reais_expenses:.2f}')
total_euros_expenses =  new_table["Total expends (e)"].sum()
print(f'The total amount expended in reais was {total_euros_expenses:.2f}')



This part of the code generates some graph to be exported to the email

In [7]:
#Import package to generate the plots
import plotly.express as px

#Iterates through all the columns ".columns" in the table "new_table"
for column in new_table.columns:
    # Creates graph using "new_table", values fo x axis "x=column", separating by colors of "Churn"
    # Examples of types of graphs are "histogram", and "barchart", press tab after "." to check the options
    graph = px.histogram(new_table, x=column, color="Gender")
    # Displays the graph
    graph.show()

The last part will summarize the tasks and send via email

In [12]:
#Import Selenium package (https://selenium-python.readthedocs.io/)
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#Import pyautogui to control the keyboard and the mouse
#To find the actual mouse position type the command 'pyautogui.position()'
import pyautogui

#Import pyperclip to copy and paste special characters
import pyperclip

#Import time to control timing
import time

#Make the code waits x seconds between each comand line
pyautogui.PAUSE = 0.5

#Open email to send information (in this case hotmail)
new_tab.get("https://outlook.live.com/mail/0/")

#Waits you to sign in in the email
print('After the log in press enter to continue')
x = input()

#Selecting the write email button
#to copy and past the pyperclip package will be used to avoid errors with special characters
xpath = '//*[@id="id__6"]'
pyperclip.copy(xpath)
new_tab.find_element_by_xpath(pyperclip.paste()).click()

#The next commands uses pyautogui to control the keyboard to send the email
#After the login you need to make sure that alt + tab goes to the email window
pyautogui.hotkey('alt','tab')

#use time to wait untill the proccess ends
time.sleep(3)
#typing the name of the recipient and choosing it (the waiting time is the proccess to save the recipient)
pyautogui.write('brunosilvadelima@hotmail.com')
pyautogui.press('tab')
time.sleep(2)

#Change to the subject scope and wrting the subject
pyautogui.press('tab')
pyautogui.write('Value of euro in reais.')
time.sleep(2)

#Changing to text scope and typing the value of the euro
pyautogui.press('tab')
email_text = f"""
Dear ******Recipient********

Each euro is worth {euro_in_reais:,.2f} reais today.
The total expenses for the day in reais were R$ {total_reais_expenses:,.2f} which corresponds to {total_euros_expenses:,.2f} euros.

Kind Regards,

********Sender*********

"""
pyperclip.copy(email_text)
pyautogui.write(pyperclip.paste())
time.sleep(3)

#Finding the right place to send the email
#pyautogui.hotkey('ctrl','enter') also works to send the e-mail
pyautogui.press('tab')
pyautogui.press('tab')
pyautogui.press('tab')
pyautogui.press('enter')

#close the window
#new_tab.quit()